In [1]:
%cd ..
%pwd
import sys
sys.path.append('./src')
from src.data_processor import DataProcessor
from src.problem_config import create_prob_config 
prob_config = create_prob_config("phase-3", "prob-2")

c:\VENV\api_prediction\monitor_train


## Load Training Data

In [42]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split

training_data = pd.read_parquet(prob_config.raw_data_path)

training_data, category_index = DataProcessor.build_category_features(
            training_data, prob_config.categorical_cols
        )
train, dev = train_test_split(training_data, test_size=0.1, random_state=123)
target_col = prob_config.target_col
train_x = training_data.drop([target_col], axis=1)
train_y = training_data[[target_col]]
test_x = dev.drop(["label"], axis=1)
test_y = dev[[target_col]]

# Store the category_index
with open(prob_config.category_index_path, "wb") as f:
    pickle.dump(category_index, f)

# Check and process saved request

In [3]:
%ls "data/captured/"

 Volume in drive C is Windows
 Volume Serial Number is 788F-AEF3

 Directory of c:\VENV\api_prediction\monitor_train\data\captured

07/11/2023  04:01 PM    <DIR>          .
07/11/2023  09:44 AM    <DIR>          ..
07/10/2023  10:32 AM        67,043,899 192.168.88.113_prob1_10_07_2023__10_32_28.pkl
07/10/2023  10:33 AM        66,963,641 192.168.88.113_prob2_10_07_2023__10_33_13.pkl
               2 File(s)    134,007,540 bytes
               2 Dir(s)  73,873,383,424 bytes free


In [4]:
saved_request = DataProcessor.load_saved_request('192.168.88.113_prob2_10_07_2023__10_33_13.pkl')
keys = list(saved_request.keys())

# Test API

In [16]:
rows = []
test = saved_request[keys[8]]

for index, row in test.iterrows():
    rows.append(row.to_list())

data = {
  "id": None,
  "rows": rows,
  "columns": test.columns.to_list()
}

import requests
#response = requests.post('http://14.225.205.204:80/phase-2/prob-1/predict', json=data)
response = requests.post('http://52.221.238.201:5040/phase-2/prob-2/predict', json=data)
#response = requests.post('http://52.221.238.201:5040/phase-2/prob-2/predict', data=data, headers={'Content-Type': 'text/plain'})
#response = requests.post('http://localhost:8000/phase-2/prob-1/predict', json=data)
print(response.text)

{"id":null,"predictions":["Normal","Information Gathering","Normal","Exploits","Exploits","Exploits","Normal","Other","Information Gathering","Exploits","Normal","Information Gathering","Normal","Normal","Information Gathering","Normal","Normal","Normal","Other","Exploits","Denial of Service","Normal","Normal","Normal","Normal","Exploits","Other","Other","Normal","Denial of Service","Denial of Service","Denial of Service","Other","Normal","Other","Normal","Other","Information Gathering","Other","Normal","Normal","Normal","Exploits","Other","Normal","Exploits","Normal","Exploits","Normal","Information Gathering","Normal","Normal","Exploits","Exploits","Denial of Service","Normal","Denial of Service","Normal","Normal","Exploits","Normal","Normal","Denial of Service","Exploits","Exploits","Other","Other","Normal","Other","Normal","Denial of Service","Information Gathering","Normal","Exploits","Exploits","Denial of Service","Normal","Exploits","Exploits","Normal","Exploits","Denial of Serv

In [48]:
import random
import asyncio
import requests

async def requests_test():
    key = random.randint(0, len(keys) - 1)
    id = random.randint(1000, 2000)
    test = saved_request[keys[key]]
    rows = []
    for index, row in test.iterrows():
        rows.append(row.to_list())
    data = {
        "id": id,
        "rows": rows,
        "columns": test.columns.to_list()
        }
    #response = requests.post('http://14.225.205.204:80/phase-2/prob-2/predict', json=data)
    response = requests.post('http://52.221.238.201:5040/phase-2/prob-2/predict', json=data)
    #response = requests.post('http://localhost:8000/phase-2/prob-2/predict', json=data)
    #print(response.text)
    return response

asyncio.create_task(requests_test())

<Task pending name='Task-9' coro=<requests_test() running at C:\Users\buikh\AppData\Local\Temp\ipykernel_43952\3787563301.py:5>>

In [ ]:
from aiocache import Cache
from aiocache.serializers import PickleSerializer

cacherequest = Cache(Cache.REDIS, endpoint="localhost", port=6379, db=1, serializer=PickleSerializer())
await cacherequest.clear()

# Load request data to DF

In [18]:
import pandas as pd
import redis
import pickle
# Load data for problem 1
rc2 = redis.Redis(host='localhost', db=2, port=6379)

captured_x = pd.DataFrame()
for key in rc2.keys():
    captured_data = pickle.loads(rc2.get(key))
    captured_x = pd.concat([captured_x, captured_data])

In [21]:
captured_x.drop_duplicates(inplace=True, ignore_index=True)

In [6]:
captured_x = pd.concat(saved_request.values(), ignore_index=True)
captured_x.drop_duplicates(inplace=True, ignore_index=True)

In [43]:
from lightgbm import LGBMClassifier
model0 = LGBMClassifier(objective="multiclass", random_state=123)#, **trial.params)
model0.fit(train_x, train_y)

c:\VENV\api_prediction\.venv\Lib\site-packages\sklearn\preprocessing\_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\VENV\api_prediction\.venv\Lib\site-packages\sklearn\preprocessing\_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


LGBMClassifier(objective='multiclass', random_state=123)

In [29]:
model0.predict(captured_x)

array(['Normal', 'Normal', 'Normal', ..., 'Other', 'Denial of Service',
       'Denial of Service'], dtype=object)

In [45]:
from sklearn.metrics import accuracy_score
predictions = model0.predict(test_x)
accuracy = accuracy_score(predictions, test_y)
metrics = {"accuracy_score": accuracy}
print(metrics)

{'accuracy_score': 0.8633791430881164}


In [33]:
train_y.value_counts()

label                
Normal                   20143
Other                    12614
Exploits                  9781
Denial of Service         8627
Information Gathering     3639
Malware                    852
Name: count, dtype: int64

In [46]:
import mlflow
import numpy as np
from mlflow.models.signature import infer_signature

desc = """
### Header
LGBM model, Tunned Model For Prob2
Model: LGBM
    """

MLFLOW_TRACKING_URI = 'http://127.0.0.1:5000'
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment("phase-2_prob-2_lgbm")
MLFLOW_MODEL_PREFIX = "model"
mlflow.start_run(description=desc)
mlflow.log_metrics(metrics)
mlflow.log_params(model0.get_params())
signature = infer_signature(test_x.astype(np.float64), predictions)
mlflow.sklearn.log_model(
    sk_model=model0,
    artifact_path=MLFLOW_MODEL_PREFIX,
    signature=signature,
    pip_requirements ='src/requirements.txt',
    registered_model_name="phase-2_prob-2_model"
)

experimentid = mlflow.active_run().info.run_id
mlflow.end_run()

c:\VENV\api_prediction\.venv\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'phase-2_prob-2_model' already exists. Creating a new version of this model...
2023/07/30 17:36:56 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: phase-2_prob-2_model, version 2
Created version '2' of model 'phase-2_prob-2_model'.


# Rejected code

In [34]:
from sklearn.cluster import MiniBatchKMeans
import joblib
import numpy as np
# N là số lượng cluster
N = 30000
# Train clustering model cho data đã có label
kmeans = MiniBatchKMeans(n_clusters=N, random_state=0, n_init='auto').fit(train_x)
joblib.dump(kmeans, 'data/phase-2/prob-2/kmeans.cpk')

['data/phase-2/prob-2/kmeans.cpk']

In [39]:
import joblib
import numpy as np

N = 30000
kmeans = joblib.load('data/phase-2/prob-2/kmeans.cpk')
new_labels = []

# Duyệt từng cluster
for  i  in  range(N):
	# Lấy các label của các data point thuộc cluster i
	mask = (kmeans.labels_ == i)
	cluster_labels = train_y[mask]

	if  len(cluster_labels) == 0:
		# Nếu cluster i rỗng thì xác định cluster i ánh xạ với 1 label mặc định (ở đây lựa chọn là 0)
		new_labels.append("Other")
	else:
		# Tìm label mới cho cả cụm cluster trong trường hợp cụm cluster khác rỗng
		#if  isinstance(train_y.flatten()[0], float):
			# Nếu là bài toán Regression thì lấy giá trị trung bình của các label thuộc cluster
		#	new_labels.append(np.mean(cluster_labels.flatten()))
		#else:
			# Nếu là bài toán Classification thì lấy label xuất hiện nhiều nhất trong cluster
			new_labels.append(cluster_labels.value_counts().idxmax()[0])

# Ánh xạ lại label cho data drift dựa trên kết quả predict cluster ở trên
y_drift_propagated = [new_labels[c] for  c  in  kmeans.labels_]

In [41]:
y_drift_test_propagated = [ new_labels[c] for c in kmeans.predict(test_x)]
from sklearn.metrics import accuracy_score
accuracy_score(y_drift_test_propagated, test_y)

0.6497978981406629

In [ ]:
# Prepair label data for training
kmeans_clusters = kmeans.predict(test_x)
#approx_label0 = [new_labels[c] for c in kmeans_clusters]
approx_label0 = yourFavoriteModel.predict(test_x)
approx_label_df0 = pd.DataFrame(approx_label0, columns=[prob_config.target_col])

# Model 0 training
train_x_new = pd.DataFrame(np.concatenate((train_x, test_x)), columns=train_x.columns)
train_y_new = pd.DataFrame(np.concatenate((train_y, approx_label_df0)), columns=train_y.columns)

In [ ]:
import featurewiz as fw

selected_features, trainm = fw.featurewiz(training_data, target=target_col, corr_limit=0.9, verbose=2)
print('Selected features:', selected_features)

In [ ]:
"""
Optuna example that optimizes a classifier configuration for cancer dataset using LightGBM.

In this example, we optimize the validation accuracy of cancer detection using LightGBM.
We optimize both the choice of booster model and their hyperparameters.

"""

import numpy as np
import optuna

import lightgbm as lgb
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split


# FYI: Objective functions can take additional arguments
# (https://optuna.readthedocs.io/en/stable/faq.html#objective-func-additional-args).
def objective(trial):
    dtrain = lgb.Dataset(train_x, label=train_y)

    param = {
        "objective": "multiclass",
        "metric": "multi_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }
    gbm = LGBMClassifier(**param)
    gbm = gbm.fit(train_x, train_y)
    preds = gbm.predict(test_x)
    accuracy = sklearn.metrics.accuracy_score(test_y, preds)
    return accuracy


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))